# Application Level Profiling

We follow a *top-down approach* to performance analysis, starting with a *whole application* performance overview and then narrowing down to specific *hot spots*.
An initial overview can be obtained using Nsight Systems, using either solely the command line interface, or by complementing the analysis with the provided GUI.

## Nsight Systems CLI

First, we compile and execute out benchmark application to make sure that results are as expected.

In [ ]:
!nvc++ -O3 -march=native -std=c++17 -mp=gpu -target=gpu ../src/stencil-2d/stencil-2d-omp-target-v0.cpp -o ../build/stencil-2d-omp-target-v0
!../build/stencil-2d-omp-target-v0 double 8192 8192 2 256

Next, we profile our binary with `nsys profile`.
Further command line arguments are:
* `--stats=true`: prints a summary of performance statistics on the command line
* `-o ...`: sets the target output profile file
* `--force-overwrite=true`: replaces the profile file if it already exists (instead of aborting)

In [ ]:
!nsys profile --stats=true -o ../profiles/stencil-2d-omp-target-v0 --force-overwrite=true ../build/stencil-2d-omp-target-v0 double 8192 8192 2 256

The output of the command line is organized in multiple categories.
A possible output for an Nvidia A40 is copied in below:

### Possible Output

```bash
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)    Med (ns)   Min (ns)  Max (ns)   StdDev (ns)           Name
 --------  ---------------  ---------  ----------  ----------  --------  ---------  -----------  ----------------------
     99.8      77348401534       3892  19873690.0  10106484.5      2394  100259663   28010088.1  poll
      0.2        117470877        547    214754.8     20558.0      1092   19384621     953829.9  ioctl
      0.0         11710433         48    243967.4      2895.5      1142   11482343    1656643.6  fopen
      0.0           886718         25     35468.7      6512.0      5450     549224     107805.2  mmap64
      0.0           807930          4    201982.5    170626.0    114085     352593     110038.8  pthread_create
      0.0           717598          9     79733.1     85691.0     34885      93486      17997.4  sem_timedwait
      0.0           172732         46      3755.0      3176.0      1072      10269       1822.6  open64
      0.0           148892         18      8271.8      6708.0      2675      27882       6152.5  mmap
      0.0           122811          1    122811.0    122811.0    122811     122811          0.0  pthread_cond_wait
      0.0            43191          1     43191.0     43191.0     43191      43191          0.0  fgets
      0.0            42830         19      2254.2      1543.0      1032       8336       2015.4  fclose
      0.0            30945          6      5157.5      2855.0      2404      10499       3872.8  munmap
      0.0            30329          7      4332.7      4369.0      1133       9288       2782.2  open
      0.0            28915          4      7228.8      6457.5      1252      14748       6793.4  fwrite
      0.0            28142          2     14071.0     14071.0      4768      23374      13156.4  fread
      0.0            26008         10      2600.8      2720.0      1623       3476        598.1  write
      0.0            24229         13      1863.8      1954.0      1062       2896        430.0  read
      0.0            16131          3      5377.0      6773.0      1914       7444       3017.8  pipe2
      0.0            11151          2      5575.5      5575.5      2946       8205       3718.7  socket
      0.0             9348          1      9348.0      9348.0      9348       9348          0.0  connect
      0.0             7614          1      7614.0      7614.0      7614       7614          0.0  putc
      0.0             5290          2      2645.0      2645.0      1703       3587       1332.2  pthread_cond_broadcast
      0.0             1243          1      1243.0      1243.0      1243       1243          0.0  bind
      0.0             1183          1      1183.0      1183.0      1183       1183          0.0  fcntl
```

```bash
[5/8] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)    Med (ns)   Min (ns)  Max (ns)  StdDev (ns)          Name
 --------  ---------------  ---------  ----------  ----------  --------  --------  -----------  --------------------
     52.9      22370448222        516  43353581.8  43790434.0  40156362  54060552    2876430.4  cuMemcpyDtoHAsync_v2
     47.0      19881247220        516  38529548.9  37066799.0  36664523  49505472    3250051.8  cuMemcpyHtoDAsync_v2
      0.0         21089569          1  21089569.0  21089569.0  21089569  21089569          0.0  cuMemAllocManaged
      0.0          9464739        258     36685.0     36944.5     24536     77786       9290.1  cuLaunchKernel
      0.0          1028562        258      3986.7      3847.0      2635      8986        956.5  cuStreamSynchronize
      0.0           880135          4    220033.8    157105.5     15529    550395     230460.4  cuMemAlloc_v2
      0.0           486545          1    486545.0    486545.0    486545    486545          0.0  cuMemAllocHost_v2
      0.0            71805          1     71805.0     71805.0     71805     71805          0.0  cuModuleLoadDataEx
      0.0             4408          4      1102.0       576.5       170      3085       1348.4  cuCtxSetCurrent
      0.0             1934          1      1934.0      1934.0      1934      1934          0.0  cuInit
```

```bash
[6/8] Executing 'cuda_gpu_kern_sum' stats report

 Time (%)  Total Time (ns)  Instances  Avg (ns)   Med (ns)   Min (ns)  Max (ns)  StdDev (ns)                     Name
 --------  ---------------  ---------  ---------  ---------  --------  --------  -----------  -------------------------------------------
    100.0       1429467171        258  5540570.4  5537425.0   5504934   5573316      15918.9  nvkernel__Z9stencil2dIdEvPKT_PS0_mm_F2L5_14
```
```bash
[7/8] Executing 'cuda_gpu_mem_time_sum' stats report

 Time (%)  Total Time (ns)  Count   Avg (ns)    Med (ns)   Min (ns)  Max (ns)  StdDev (ns)           Operation
 --------  ---------------  -----  ----------  ----------  --------  --------  -----------  ----------------------------
     51.3      20899546153    516  40502996.4  40379936.0  40074517  48481627     472914.7  [CUDA memcpy Device-to-Host]
     48.7      19866014386    516  38500027.9  37032058.0  36637994  49470798    3250287.5  [CUDA memcpy Host-to-Device]
```


```bash
[8/8] Executing 'cuda_gpu_mem_size_sum' stats report

 Total (MB)  Count  Avg (MB)  Med (MB)  Min (MB)  Max (MB)  StdDev (MB)           Operation
 ----------  -----  --------  --------  --------  --------  -----------  ----------------------------
 277025.391    516   536.871   536.871   536.871   536.871        0.000  [CUDA memcpy Device-to-Host]
 277025.391    516   536.871   536.871   536.871   536.871        0.000  [CUDA memcpy Host-to-Device]
```

### Exercise

Look at the performance data and try to answer the following questions:
* What is the largest execution time contributor?
* How often is the main kernel launched?
* Is synchronization applied after every kernel launch?
* Does the time spent in synchronization match your expectation?
* Do transfer sizes match your expectation?

### Solution

Looking at the statistics we can see multiple effects:
* The number of kernel instances matches our expectation ($2 + 256$).
* The number of memory transfers seems to be related to the number of kernel instances.
* Comparing the time spent in GPU synchronization (`cuStreamSynchronize`) and kernel execution time shows a mismatch.
* Comparing aggregated memory transfer and kernel execution times reveals an order of magnitude in difference.
  * \> Even if the kernel could be accelerated, overall performance will most likely not increase.
  * \> These numbers could be used to approximate a minimum number of iterations at which the memory transfers get amortized (assuming that the transfer times *don't scale with the number of iterations*).
* The size per transfer matches our expectation (8192**2 $\cdot$ 8 B $\approx$ 537 MB).

## Nsight Systems GUI

Next, we further investigate the problematic memory transfers by opening up the generated `stencil-2d-omp-target-v0` report file which is in the `../profiles` folder.

### Exercise

Download the produced file and open it with your local installation of Nsight Systems.
Can you connect the timeline to some of the effects seen previously?

### Solution

As maybe already suspected, the timeline shows a recurring pattern of
* two memory transfers (HtoD),
* a kernel call, and
* two memory transfers (DtoH)

Additionally, `cudaStreamSynchronize` is only called at the end of this pattern, which partly explains the deviation from the kernel execution time.

You might also be wondering why the asynchronous data transfers show up as synchronous.
This is an artefact of the way the host memory was allocated - pageable instead of pinned (or page-locked).
Using different allocators, as e.g. `cudaMallocHost`, this could be changed.
Without pinned memory, the CUDA runtime is required to stage memory transfers via a pinned buffer which has two effects:
* transfers are done synchronously and
* transfer rates are lower than expected (compare the achieved $\thicksim 13 \text{GB/s}$ with the theoretical maximum of $31.5 \text{GB/s}$).

## Stencil Code Optimization 1

Having pinpointed our performance bug, we can now optimize data transfers in our application.
One straight-forward way is adding unstructured data primitives in our code, basically spanning a region at whose begin and end data is copied *only one time*.
The updated version is available at [stencil-2d-omp-target-v1.cpp](../src/stencil-2d/stencil-2d-omp-target-v1.cpp), and can be compiled, executed and profiled using the following cells.

In [ ]:
!nvc++ -O3 -march=native -std=c++17 -mp=gpu -target=gpu ../src/stencil-2d/stencil-2d-omp-target-v1.cpp -o ../build/stencil-2d-omp-target-v1

In [ ]:
!../build/stencil-2d-omp-target-v1 double 8192 8192 2 256

In [ ]:
!nsys profile --stats=true -o ../profiles/stencil-2d-omp-target-v1 --force-overwrite=true ../build/stencil-2d-omp-target-v1 double 8192 8192 2 256

### Possible Output

```bash
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)    Med (ns)   Min (ns)  Max (ns)   StdDev (ns)           Name
 --------  ---------------  ---------  ----------  ----------  --------  ---------  -----------  ----------------------
     96.0       3043642947        159  19142408.5  10106826.0      2034  100171361   27435826.4  poll
      3.5        111183639        549    202520.3     23104.0      1032   19495430     973186.6  ioctl
      0.4         12125839         47    257996.6      2785.0      1022   11922763    1738477.2  fopen
      0.0           914969         25     36598.8      8215.0      5710     501774      98386.9  mmap64
      0.0           809294          9     89921.6     88938.0     79540     106941      10396.3  sem_timedwait
      0.0           740053          4    185013.3    189692.0    145544     215125      31422.4  pthread_create
      0.0           178407         45      3964.6      3457.0      1002       7845       1618.1  open64
      0.0           163398         18      9077.7      6963.0      3868      31770       6838.4  mmap
      0.0           121709          1    121709.0    121709.0    121709     121709          0.0  pthread_cond_wait
      0.0            40298         19      2120.9      1332.0      1052       8586       2121.0  fclose
      0.0            37610          1     37610.0     37610.0     37610      37610          0.0  fgets
      0.0            34245          6      5707.5      5139.5      2916       8616       2158.8  open
      0.0            33272          6      5545.3      4839.0      2815       9357       2490.0  munmap
      0.0            32101          2     16050.5     16050.5      5741      26360      14579.8  fread
      0.0            27362         10      2736.2      2369.5      1774       4929       1113.1  write
      0.0            24718         12      2059.8      1968.5      1533       2956        410.0  read
      0.0            22322          5      4464.4      2094.0      1202       9227       4073.0  fwrite
      0.0            15379          3      5126.3      6041.0      2134       7204       2655.9  pipe2
      0.0            14648          2      7324.0      7324.0      6052       8596       1798.9  socket
      0.0            11202          1     11202.0     11202.0     11202      11202          0.0  connect
      0.0             7955          1      7955.0      7955.0      7955       7955          0.0  putc
      0.0             6452          2      3226.0      3226.0      1343       5109       2663.0  pthread_cond_broadcast
      0.0             2354          2      1177.0      1177.0      1042       1312        190.9  fcntl
      0.0             2124          1      2124.0      2124.0      2124       2124          0.0  bind
```

```bash
[5/8] Executing 'cuda_api_sum' stats report

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)    Med (ns)   Min (ns)  Max (ns)  StdDev (ns)          Name
 --------  ---------------  ---------  ----------  ----------  --------  --------  -----------  --------------------
     88.8       1431094155        260   5504208.3   5545909.5      1253   5662444     485051.2  cuStreamSynchronize
      5.1         81482718          2  40741359.0  40741359.0  40733549  40749169      11045.0  cuMemcpyDtoHAsync_v2
      4.6         74672375          2  37336187.5  37336187.5  36930684  37741691     573468.5  cuMemcpyHtoDAsync_v2
      1.3         21050616          1  21050616.0  21050616.0  21050616  21050616          0.0  cuMemAllocManaged
      0.1          1085022        258      4205.5      3837.0      3497     39664       2436.0  cuLaunchKernel
      0.0           782233          4    195558.3    185644.5      8196    402748     168722.5  cuMemAlloc_v2
      0.0           494700          1    494700.0    494700.0    494700    494700          0.0  cuMemAllocHost_v2
      0.0           123833          1    123833.0    123833.0    123833    123833          0.0  cuModuleLoadDataEx
      0.0             2634          4       658.5       671.0       190      1102        372.6  cuCtxSetCurrent
      0.0              992          1       992.0       992.0       992       992          0.0  cuInit
```

```bash
[6/8] Executing 'cuda_gpu_kern_sum' stats report

 Time (%)  Total Time (ns)  Instances  Avg (ns)   Med (ns)   Min (ns)  Max (ns)  StdDev (ns)                     Name
 --------  ---------------  ---------  ---------  ---------  --------  --------  -----------  -------------------------------------------
    100.0       1430532679        258  5544700.3  5544388.0   5508578   5598691      18042.4  nvkernel__Z9stencil2dIdEvPKT_PS0_mm_F2L5_14
```

```bash
[7/8] Executing 'cuda_gpu_mem_time_sum' stats report

 Time (%)  Total Time (ns)  Count   Avg (ns)    Med (ns)   Min (ns)  Max (ns)  StdDev (ns)           Operation
 --------  ---------------  -----  ----------  ----------  --------  --------  -----------  ----------------------------
     52.1         81029570      2  40514785.0  40514785.0  40396288  40633282     167580.1  [CUDA memcpy Device-to-Host]
     47.9         74573978      2  37286989.0  37286989.0  36897514  37676464     550800.8  [CUDA memcpy Host-to-Device]
```

```bash
[8/8] Executing 'cuda_gpu_mem_size_sum' stats report

 Total (MB)  Count  Avg (MB)  Med (MB)  Min (MB)  Max (MB)  StdDev (MB)           Operation
 ----------  -----  --------  --------  --------  --------  -----------  ----------------------------
   1073.742      2   536.871   536.871   536.871   536.871        0.000  [CUDA memcpy Device-to-Host]
   1073.742      2   536.871   536.871   536.871   536.871        0.000  [CUDA memcpy Host-to-Device]
```

### Exercise

Revisit the observations from before.
Which of them still hold, which are now different?

Also visualize the time line - are the code changes reflected as expected?

### Solution

Looking at the statistics we can now compare the effects previously discussed:
* The number of kernel instances matches our expectation ($2 + 256$).
* ~~The number of memory transfers seems to be related to the number of kernel instances.~~
* ~~Comparing the time spent in GPU synchronization (`cuStreamSynchronize`) and kernel execution time shows a mismatch.~~
* ~~Comparing aggregated memory transfer and kernel execution times reveals an order of magnitude in difference.~~
* The size per transfer matches our expectation ($8192 \cdot 8192 \cdot 8 \text{B} \approx 537 \text{MB}$).

Opening up the generated profile output in our GUI reveals what we already expected: single staging parts at the beginning and the end of our application with multiple kernel calls in between.
The output file is once again collected in the `../profiles` folder.

## Next Step

The main execution time contributor is now the kernel execution.
Since we have only a single kernel, further hot spot analysis is not necessary in this case.
Instead, we directly focus on its performance in the [Kernel Level Profiling](./kernel-level-profiling.ipynb) notebook.